In [4]:
from datasets import load_dataset
import datasets
from collections import Counter
import re
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import tokenizers
import pandas as pd
from torch.utils.data import Dataset, DataLoader
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 加载 IMDB 数据集
cache_dir=r'.\data_file'
imdb_dataset = load_dataset("imdb",cache_dir=cache_dir)
# .map(): 对所有样本应用一个函数 (非常重要，用于预处理):

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at data_file\imdb\plain_text\0.0.0\e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Thu Jun  5 21:25:29 2025).


In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, data_type):
        super(IMDBDataset, self).__init__()
        self.dataset=imdb_dataset[data_type]
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

In [6]:
imdb_data=IMDBDataset('train')

In [9]:
imdb_data['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [10]:
import nltk

In [13]:
import spacy

# 1. 加载预训练模型
# 确保模型包含了 'senter' (推荐 for spaCy v3.0+) 或 'parser' 组件
# 'en_core_web_sm' 默认包含这些组件，所以可以直接用
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [18]:
import spacy
import sys
import subprocess

def download_spacy_model(model_name):
    """
    在 Python 中下载 spaCy 模型。
    如果模型已经存在，则不会重复下载。
    """
    try:
        spacy.load(model_name)
        print(f"'{model_name}' 模型已存在，无需下载。")
    except OSError:
        print(f"正在下载 '{model_name}' 模型...")
        try:
            # 使用 subprocess 运行命令行下载命令
            # 这种方法更健壮，因为它直接调用 spacy 的 CLI 逻辑
            subprocess.check_call([sys.executable, "-m", "spacy", "download", model_name])
            print(f"'{model_name}' 模型下载完成。")
        except subprocess.CalledProcessError as e:
            print(f"下载 '{model_name}' 模型失败: {e}")
            print("请尝试手动运行 'python -m spacy download {}'".format(model_name))
        except Exception as e:
            print(f"发生未知错误: {e}")

# 要下载的模型名称
model_to_download = "en_core_web_sm"

# 调用函数下载模型
download_spacy_model(model_to_download)

# 现在你可以安全地加载和使用模型了
try:
    nlp = spacy.load(model_to_download)
    print(f"\n'{model_to_download}' 模型加载成功！可以开始使用了。")
    doc = nlp("Hello, how are you? This is a test sentence.")
    for sent in doc.sents:
        print(sent.text)
except Exception as e:
    print(f"加载模型失败: {e}")



正在下载 'en_core_web_sm' 模型...
'en_core_web_sm' 模型下载完成。

'en_core_web_sm' 模型加载成功！可以开始使用了。
Hello, how are you?
This is a test sentence.


In [23]:
nlp = spacy.load(model_to_download)
doc = nlp(imdb_data['text'][0])

In [24]:
for sent in doc.sents:
    print(sent.text)

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967.
I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life.
In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States.
In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br
/>What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic.
Really, the sex and nudity scenes are few and far between, eve